In [1]:
import gc
gc.collect()

import sys
sys.path.insert(0, '../')
import logging
logging.basicConfig(level=logging.ERROR)

import pandas as pd
import numpy as np
from datetime import datetime, timedelta

from cryptotrader.exchange_api.poloniex import Poloniex
from cryptotrader.envs.trading import BacktestDataFeed, BacktestEnvironment
from cryptotrader.envs.utils import make_balance, convert_to
from cryptotrader.agents import apriori
from cryptotrader.utils import array_normalize, simplex_proj

from time import time, sleep

from bokeh.io import output_notebook
output_notebook()
%matplotlib inline
%load_ext line_profiler

Loading BokehJS ...

In [2]:
# Environment setup
obs_steps = 300
period = 30
pairs = ["USDT_BTC", "USDT_ETH", "USDT_LTC", "USDT_XRP", "USDT_XMR", "USDT_ETC", "USDT_DASH"]
init_funds = make_balance(crypto=0.0, fiat=100.0, pairs=pairs)

tapi = Poloniex()
tapi = BacktestDataFeed(tapi, period, './data',pairs, init_funds)
# tapi.download_data(end=datetime.timestamp(datetime.utcnow() - timedelta(days=100)),
#                        start=datetime.timestamp(datetime.utcnow() - timedelta(days=200)))
tapi.load_data('/train')

env = BacktestEnvironment(period, obs_steps, tapi, 'backtest_env')
env.add_pairs(pairs)
env.fiat = 'USDT'
obs = env.reset()

INFO:backtest_env:[Trading Environment initialization]
Trading Environment Initialized!



In [3]:
# Training run
# Training params
nb_steps = 50
batch_size = 2
nb_max_episode_steps = 7

agent = apriori.MomentumTrader(ma_span=[3,50], std_span=50, activation=simplex_proj)

# hp = {
#     'ma1': [2, env.obs_steps],
#     'ma2': [2, env.obs_steps],
#     'std_span': [2, env.obs_steps],
#     'alpha_up': [1e-4, 5e-1],
#     'alpha_down': [1e-4, 5e-1]
#     }

# search_space = {'mean_type':{'simple': hp,
#                              'exp': hp,
#                              'kama': hp
#                              }
#                 }
# constrains = [lambda mean_type, ma1, ma2, std_span, alpha_up, alpha_down: ma1 < ma2]

# opt_params, info = agent.fit(env, nb_steps, batch_size, search_space, constrains, nb_max_episode_steps=nb_max_episode_steps)
# print("\n", opt_params,"\n", env.status)

# Previous found params:
# {'mean_type': 'kama', 'alpha_down': 0.4128425807825884, 'alpha_up': 0.37140222586733046, 'ma1': 133, 'ma2': 234, 'std_span': 39}

agent.test(env, verbose=True)
env.plot_results();

>> step 332/4498, 7 % done, Cumulative Reward: 1.1522619807286232, ETC: 0 days 00:09:46.513202, Samples/s: 7.0990                        
Keyboard Interrupt: Stoping backtest
Elapsed steps: 332/4800, 6 % done.

################### > Portifolio Performance Analysis < ###################

Portifolio excess Sharpe:                 -0.155972
Portifolio / Benchmark Sharpe ratio:      -0.638830 / 2.830434
Portifolio / Benchmark Omega ratio:       0.877525 / 1.658871
Portifolio / Benchmark max drawdown:      -13.174159 / -4.918161


In [4]:
# Validation run
# tapi.download_data(end=datetime.timestamp(datetime.now() - timedelta(days=50)),
#                        start=datetime.timestamp(datetime.now() - timedelta(days=100)))
env.tapi.load_data('/eval')
agent.test(env, verbose=True)
env.plot_results();

>> step 76/2098, 3 % done, Cumulative Reward: 1.017850290121508, ETC: 0 days 00:04:41.584311, Samples/s: 7.1762                         
Keyboard Interrupt: Stoping backtest
Elapsed steps: 76/2400, 3 % done.

################### > Portifolio Performance Analysis < ###################

Portifolio excess Sharpe:                 -0.340022
Portifolio / Benchmark Sharpe ratio:      -1.929457 / 2.878540
Portifolio / Benchmark Omega ratio:       0.675033 / 1.570209
Portifolio / Benchmark max drawdown:      -1.247952 / -2.062129


In [5]:
# Test run
# tapi.download_data(end=datetime.timestamp(datetime.now()),
#                        start=datetime.timestamp(datetime.now() - timedelta(days=50)))
env.tapi.load_data('/test')
agent.test(env, verbose=True)
env.plot_results();

>> step 31/2096, 1 % done, Cumulative Reward: 0.9635752970129188, ETC: 0 days 00:04:44.222660, Samples/s: 7.2611                        
Keyboard Interrupt: Stoping backtest
Elapsed steps: 31/2398, 1 % done.

################### > Portifolio Performance Analysis < ###################

Portifolio excess Sharpe:                 -0.188067
Portifolio / Benchmark Sharpe ratio:      -5.600369 / -3.167157
Portifolio / Benchmark Omega ratio:       0.266291 / 0.565188
Portifolio / Benchmark max drawdown:      -1.194545 / -1.679504
